In [16]:
%reload_ext autoreload
%autoreload 2
# imports
import time
import os
import pandas as pd
import re
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


In [2]:
# Open the LinkedIn
driver = webdriver.Chrome()

# Navigate to Jobs Page
try:
    job_title = "Data Scientist" 
    job_location ="United States"
    page_num=0
    url = f'https://www.linkedin.com/jobs/search/?keywords={job_title}&location={job_location}&start={page_num}'
    driver.get(url)
    driver.maximize_window()
except Exception as e:
    print(e)

while True:    
    total_jobs = len(driver.find_elements(By.CSS_SELECTOR, "ul.jobs-search__results-list li"))
    # check if the total jobs shown is over 500
    if total_jobs >= 200:
        break
    
    # perform scrolling activity
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    
    try:
        # click the see more jobs if its shown
        next=driver.find_element(By.XPATH, "//button[@aria-label='See more jobs']")
        next.click()
        time.sleep(2)
    except:
        pass

jobs  = driver.find_elements(By.CSS_SELECTOR, "ul.jobs-search__results-list li")
job_titles = []
job_posteds = []
job_descriptions = []
job_functions = []
job_industries = []
print(len(jobs))
for i, job in enumerate(jobs):
    try:
        print(i, end="\r")
        job.click()
        time.sleep(3)
        see_more_button = WebDriverWait(driver, 100).until( 
            EC.presence_of_element_located((By.XPATH, '//button[@aria-label="Show more, visually expands previously read content above"]')) 
        )
        see_more_button.click()
        time.sleep(1)
        
        soup = BeautifulSoup(driver.page_source, "html.parser")
        job_title_element = soup.find('h2', {'class':'top-card-layout__title'})
        if job_title_element is None:
            job_titles.append("Not Found")
        else:
            job_titles.append(job_title_element.text.strip())
        
        job_posted_element = soup.find('span', {'class':'posted-time-ago__text'})
        if job_posted_element is None:
            job_posteds.append("Not Found")
        else:
            job_posteds.append(job_posted_element.text.strip())
            
        job_description_element = soup.find('div', {'class':'show-more-less-html__markup relative overflow-hidden'})        
        if job_description_element is None:
            job_descriptions.append("Not Found")
        else:
            job_description = re.sub("<[^>]+>", " ", str(job_description_element.text))
            job_description = re.sub("[ ]+", " ", job_description)
            job_descriptions.append(job_description)
            
        job_function = soup.select("ul.description__job-criteria-list > li:nth-child(3) > span")
        if job_function is None:
            job_functions.append("Not Found")
        else:
            job_functions.append(job_function[0].text.strip())
            
        job_industry = soup.select("ul.description__job-criteria-list > li:nth-child(4) > span")
        if job_industry is None:
            job_industries.append("Not Found")
        else:
            job_industries.append(job_industry[0].text.strip())
        print(f"{i}: {job_title_element.text.strip()}, {job_posted_element.text.strip()}, {job_description_element.text.strip()}, {job_function[0].text.strip()}, {job_industry[0].text.strip()}")
    except Exception as e:
        print(f"At iteration {i} with error of {e}")
    break

223
0: Junior Data Scientist, 7 hours ago, This is a remote position. Junior Data Scientist (1 year experience, remote)Be part of our future! This job posting builds our talent pool for potential future openings. We'll compare your skills and experience against both current and future needs. If there's a match, we'll contact you directly. No guarantee of immediate placement, and we only consider applications from US/Canada residents during the application process. Hiring Type: Full-Time Base Salary: $60K-$70K Per Annum.Responsibilities:Partner with engineers, product managers, and business partners to identify algorithmic problems, brainstorm possible approaches, and recommend the best path forward. Develop algorithms iteratively, building in the right level of complexity to solve the business problem at hand and support future improvements. Define success criteria for your models so that you can measure impact and changes over time. You'll be expected to communicate findings and drive

In [5]:
pd.DataFrame(
    {
        "job_title": job_titles,
        "job_posted": job_posteds,
        "job_description": [job_description_element],
        "job_function": job_functions,
        "job_industry": job_industries,
    }
)
# .to_csv("../data/raw/linkedin_jobs.csv", index=False)

,job_title,job_posted,job_description,job_function,job_industry
0,Junior Data Scientist,7 hours ago,"[\n This is a remote position., [], [],...",Information Technology,Software Development


In [23]:
import re
# The regex helps in detecting html tags
# 1. The [^>] mean that it matches any character except '>'
text = re.sub("<[^>]+>", " ", str(job_description_element)).strip()
re.sub("[ ]+", " ", text)

"This is a remote position. Junior Data Scientist (1 year experience, remote) Be part of our future! This job posting builds our talent pool for potential future openings. We'll compare your skills and experience against both current and future needs. If there's a match, we'll contact you directly. No guarantee of immediate placement, and we only consider applications from US/Canada residents during the application process. Hiring Type: Full-Time Base Salary: $60K-$70K Per Annum. Responsibilities: Partner with engineers, product managers, and business partners to identify algorithmic problems, brainstorm possible approaches, and recommend the best path forward. Develop algorithms iteratively, building in the right level of complexity to solve the business problem at hand and support future improvements. Define success criteria for your models so that you can measure impact and changes over time. You'll be expected to communicate findings and drive continuous improvements. Collaborate w